In [23]:
import os
import re
from typing import TypedDict, Optional, Dict, Any
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")

from langchain_community.tools import ShellTool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, START

load_dotenv()

True

In [24]:
# --- 1. 상태 정의 ---
class HostCheckState(TypedDict):
    target_ip: str  # 입력: 확인할 IP 주소
    check_intent: str # 입력: 사용자의 확인 의도 (예: "3번만 ping 보내줘")
    generated_command_raw: Optional[str] # LLM이 생성한 원본 명령어
    ping_command: Optional[str]  # 검증된 ping 명령어
    raw_ping_output: Optional[str]  # ping 실행의 원시 출력
    is_alive: Optional[bool] # 호스트 생존 여부
    ping_summary: Optional[str] # ping 결과 요약
    error_message: Optional[str]  # 오류 발생 시 메시지 저장

In [25]:
# --- 2. LLM 및 관련 구성 요소 초기화 (명령어 생성용) ---
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
except Exception as e:
    print(f"ChatOpenAI 초기화 중 오류: {e}. API 키가 올바르게 설정되었는지 확인하세요.")
    llm = None

# 🚨 이 프롬프트는 LLM의 출력을 제어하고 안전을 확보하기 위한 매우 중요한 부분입니다.
PING_COMMAND_GENERATION_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 네트워크 유틸리티 전문가입니다. 사용자의 의도와 대상 IP를 기반으로 'ping' 명령어를 생성해야 합니다. "
     "생성된 명령어는 반드시 'ping'으로 시작해야 합니다. "
     "이 명령어는 호스트의 응답 여부를 확인하는 목적으로만 사용되어야 합니다. "
     "기본적으로 Linux 환경을 가정하여 명령어를 생성해주세요 (예: 'ping -c <횟수> <대상 IP>'). "
     "사용자 의도에 횟수나 간격 등의 조건이 있다면 반영하되, 없다면 기본값(예: 3-4회)을 사용하세요. "
     "대상 IP 주소는 명령어에 정확하게 포함되어야 합니다. "
     "출력은 오직 생성된 ping 명령어 문자열만이어야 하며, 다른 설명이나 추가 텍스트는 포함하지 마십시오. "
     "만약 사용자의 의도가 불분명하거나 안전하지 않다고 판단되면, 명령어 대신 'Error: Intent unclear or potentially unsafe.' 라고 정확히 응답하십시오."),
    ("human", "대상 IP: {target_ip}\n확인 의도: {check_intent}\n\n생성할 Ping 명령어:")
])

if llm:
    ping_command_generation_chain = PING_COMMAND_GENERATION_PROMPT_TEMPLATE | llm | StrOutputParser()
else:
    def fallback_ping_chain(_):
        return "Error: LLM not initialized. Cannot generate ping command."
    ping_command_generation_chain = fallback_ping_chain

In [26]:
ping_command_generation_chain

ChatPromptTemplate(input_variables=['check_intent', 'target_ip'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="당신은 네트워크 유틸리티 전문가입니다. 사용자의 의도와 대상 IP를 기반으로 'ping' 명령어를 생성해야 합니다. 생성된 명령어는 반드시 'ping'으로 시작해야 합니다. 이 명령어는 호스트의 응답 여부를 확인하는 목적으로만 사용되어야 합니다. 기본적으로 Linux 환경을 가정하여 명령어를 생성해주세요 (예: 'ping -c <횟수> <대상 IP>'). 사용자 의도에 횟수나 간격 등의 조건이 있다면 반영하되, 없다면 기본값(예: 3-4회)을 사용하세요. 대상 IP 주소는 명령어에 정확하게 포함되어야 합니다. 출력은 오직 생성된 ping 명령어 문자열만이어야 하며, 다른 설명이나 추가 텍스트는 포함하지 마십시오. 만약 사용자의 의도가 불분명하거나 안전하지 않다고 판단되면, 명령어 대신 'Error: Intent unclear or potentially unsafe.' 라고 정확히 응답하십시오."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['check_intent', 'target_ip'], input_types={}, partial_variables={}, template='대상 IP: {target_ip}\n확인 의도: {check_intent}\n\n생성할 Ping 명령어:'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Complet

In [27]:
# --- 3. 노드 함수 정의 ---

def generate_and_validate_ping_command_node(state: HostCheckState) -> HostCheckState:
    print("--- NODE: Ping 명령어 생성 및 검증 중... ---")
    target_ip = state.get("target_ip")
    check_intent = state.get("check_intent")
    updates: Dict[str, Optional[str]] = {"error_message": None}

    if not target_ip or not check_intent:
        updates["error_message"] = "오류: 대상 IP 또는 확인 의도가 제공되지 않았습니다."
        return updates

    try:
        print(f"LLM에게 Ping 명령어 생성 요청 (IP: {target_ip}, 의도: {check_intent})")
        generated_command_str = ping_command_generation_chain.invoke({
            "target_ip": target_ip,
            "check_intent": check_intent
        })
        updates["generated_command_raw"] = generated_command_str
        print(f"LLM 생성 원본 명령어: '{generated_command_str}'")

        stripped_command = generated_command_str.strip()

        if stripped_command.startswith("Error:"):
            raise ValueError(f"LLM 오류 반환: {stripped_command}")

        if not stripped_command.startswith("ping "): # 'ping' 뒤에 공백 중요
            raise ValueError("생성된 명령어가 'ping '으로 시작하지 않습니다.")

        if target_ip not in stripped_command:
            # IP 주소 검증 강화 필요 (예: 정규식)
            print(f"경고: 대상 IP({target_ip})가 생성된 명령어에 문자열 그대로 포함되지 않았을 수 있습니다. 명령어: '{stripped_command}'")

        # 기본적인 위험 키워드 필터링 (ping 명령어는 상대적으로 옵션이 적지만, 여전히 주의)
        forbidden_keywords = [";", "&&", "||", "`", "$(", "sudo", "rm ", "mv ", "cp ", ">", "<", "|&", "nmap", "nc ", "wget", "curl"]
        for keyword in forbidden_keywords:
            if keyword in stripped_command:
                raise ValueError(f"명령어에 허용되지 않는 키워드 '{keyword}' 포함됨.")
        
        # ping 명령어의 옵션은 비교적 안전하지만, 과도한 패킷 크기나 요청률을 유도하는 옵션은 주의
        # 예: -s (패킷 크기)가 너무 크거나, -i (간격)이 너무 짧은 경우 등
        # 이 예제에서는 상세한 옵션 값 검증은 생략합니다.

        print(f"기본 검증 통과: '{stripped_command}'")
        updates["ping_command"] = stripped_command

    except Exception as e:
        print(f"Ping 명령어 생성/검증 중 오류: {e}")
        updates["error_message"] = f"Ping 명령어 생성/검증 오류: {str(e)}"
    
    return updates

In [28]:
def execute_ping_node(state: HostCheckState) -> HostCheckState:
    print("--- NODE: Ping 실행 중... ---")
    updates: Dict[str, Optional[str]] = {"error_message": None}
    if state.get("error_message"):
        print(f"이전 단계 오류로 Ping 실행 중단: {state.get('error_message')}")
        return {}

    ping_command = state.get("ping_command")
    if not ping_command:
        updates["error_message"] = "오류: 실행할 Ping 명령어가 없습니다."
        return updates

    shell_tool = ShellTool()
    try:
        print(f"실행 명령어: {ping_command}")
        raw_output = shell_tool.invoke(ping_command)
        updates["raw_ping_output"] = raw_output
    except Exception as e:
        print(f"Ping 실행 중 오류: {e}")
        updates["error_message"] = f"Ping 실행 중 오류 발생: {str(e)}"
    
    return updates

In [29]:
def parse_ping_results_node(state: HostCheckState) -> HostCheckState:
    print("--- NODE: Ping 결과 파싱 중... ---")
    updates: Dict[str, Any] = {"is_alive": False, "ping_summary": "No response or failed to parse.", "error_message": state.get("error_message")} # 기본값 설정
    
    if state.get("error_message") and not state.get("raw_ping_output"):
        print(f"이전 단계 오류 또는 결과 없음으로 파싱 중단: {state.get('error_message')}")
        return updates # 이미 설정된 에러 메시지 유지하며 is_alive=False

    raw_output = state.get("raw_ping_output")
    if not raw_output:
        updates["error_message"] = "오류: 파싱할 Ping 결과가 없습니다."
        return updates

    # Linux ping 출력 기준 파싱 (간단화된 버전)
    # 성공 예: "0% packet loss" 또는 "bytes from" 포함, time=...
    # 실패 예: "100% packet loss" 또는 "Destination Host Unreachable"
    
    # 좀 더 관대한 성공/실패 조건 설정
    if "bytes from" in raw_output.lower() and "time=" in raw_output.lower() and "0% packet loss" in raw_output.lower(): # 가장 확실한 성공
        updates["is_alive"] = True
        updates["ping_summary"] = "Host is alive (0% packet loss)."
    elif "bytes from" in raw_output.lower() and "time=" in raw_output.lower(): # 일부 패킷 손실 있어도 응답은 온 경우
        updates["is_alive"] = True
        loss_match = re.search(r"(\d+)% packet loss", raw_output)
        loss_percent = loss_match.group(1) if loss_match else "unknown"
        updates["ping_summary"] = f"Host is alive (responded, {loss_percent}% packet loss)."
    elif "0 received" in raw_output.lower() or "100% packet loss" in raw_output.lower():
        updates["is_alive"] = False
        updates["ping_summary"] = "Host is NOT alive (100% packet loss or 0 received)."
    elif "destination host unreachable" in raw_output.lower():
        updates["is_alive"] = False
        updates["ping_summary"] = "Host is NOT alive (Destination Host Unreachable)."
    elif "name or service not known" in raw_output.lower():
        updates["is_alive"] = False
        updates["ping_summary"] = "Host name or service not known."
        updates["error_message"] = "DNS resolution failed or invalid hostname." # 추가 에러 정보
    else:
        # 위 조건에 맞지 않는 경우, 원본 출력의 일부를 요약으로 사용
        summary_lines = raw_output.splitlines()
        # 마지막 몇 줄 (통계 정보가 있을 수 있음) 또는 주요 메시지
        if len(summary_lines) > 3:
            updates["ping_summary"] = "\n".join(summary_lines[-3:])
        else:
            updates["ping_summary"] = raw_output[:200] # 처음 200자
        # is_alive는 불확실하므로 False 유지하거나, 추가적인 분석 필요
        print(f"Ping 결과가 명확하지 않음. 원본 출력: {raw_output[:100]}...")

    print(f"파싱 결과: is_alive={updates['is_alive']}, summary='{updates['ping_summary']}'")
    return updates

In [30]:
# --- 4. 그래프 빌드 ---
workflow = StateGraph(HostCheckState)

workflow.add_node("generate_and_validate_ping_command", generate_and_validate_ping_command_node)
workflow.add_node("execute_ping", execute_ping_node)
workflow.add_node("parse_ping_results", parse_ping_results_node)

workflow.set_entry_point("generate_and_validate_ping_command")

In [31]:
def decide_after_command_gen(state: HostCheckState) -> str:
    if state.get("error_message") or not state.get("ping_command"):
        print("--- 조건: Ping 명령어 생성/검증 실패 또는 명령어 없음, 종료 ---")
        return END
    else:
        print("--- 조건: Ping 명령어 생성/검증 성공, Ping 실행으로 ---")
        return "execute_ping"

workflow.add_conditional_edges(
    "generate_and_validate_ping_command",
    decide_after_command_gen,
    {
        "execute_ping": "execute_ping",
        END: END
    }
)

In [32]:
def decide_after_ping_execution(state: HostCheckState) -> str:
    if state.get("error_message") and not state.get("raw_ping_output"): # 실행 중 에러 발생
        print(f"--- 조건: Ping 실행 실패 ({state.get('error_message')}), 종료 ---")
        return END
    else: # 실행은 되었으나 결과가 없을 수도 있음 (예: 타임아웃)
        print("--- 조건: Ping 실행 완료 (성공/실패 무관), 결과 파싱으로 ---")
        return "parse_ping_results"

workflow.add_conditional_edges(
    "execute_ping",
    decide_after_ping_execution,
    {
        "parse_ping_results": "parse_ping_results",
        END: END
    }
)

workflow.add_edge("parse_ping_results", END)

In [33]:
app = workflow.compile()

In [ ]:
# target_ip_to_check = "127.0.0.1"  # 로컬호스트
target_ip_to_check = "8.8.8.8"  # Google DNS (일반적으로 ping 응답)
# target_ip_to_check = "192.0.2.1" # 응답하지 않을 가능성이 높은 테스트 IP (TEST-NET-1)
# target_ip_to_check = "nonexistenthostname12345.com" # DNS 실패 유도

check_intent_for_llm = "이 호스트가 살아있는지 확인하기 위해 3번만 ping을 보내줘."
# check_intent_for_llm = "빠르게 한번만 ping 보내줘."

print(f"📢 {target_ip_to_check}에 대한 Ping 테스트를 시작합니다...")
print(f"   확인 의도: {check_intent_for_llm}")
print("---")

initial_state = {
    "target_ip": target_ip_to_check,
    "check_intent": check_intent_for_llm
}

if not os.getenv("OPENAI_API_KEY") or not llm:
    print("OPENAI_API_KEY가 설정되지 않았거나 LLM 초기화에 실패하여 테스트를 실행할 수 없습니다.")
else:
    try:
        final_state = app.invoke(initial_state, {"recursion_limit": 10})

        print("\n---")
        print("📊 최종 Ping 테스트 결과:")
        current_error = final_state.get("error_message")
        if current_error: # 최종 상태의 에러 메시지 (어느 단계에서든 발생 가능)
            print(f"❌ 작업 중 오류 발생: {current_error}")
        
        print(f"  - 대상 IP: {final_state.get('target_ip')}")
        print(f"  - 확인 의도: {final_state.get('check_intent')}")
        print(f"  - LLM 생성 원본 명령어: {final_state.get('generated_command_raw')}")
        print(f"  - 실행된 (검증된) 명령어: {final_state.get('ping_command')}")
        
        is_alive = final_state.get('is_alive')
        if is_alive is not None:
            print(f"  - 호스트 생존 여부: {'🟢 살아있음 (응답)' if is_alive else '🔴 응답 없음 / 실패'}")
        else:
            print("  - 호스트 생존 여부: ⚠️ 판단 불가 (결과 파싱 실패 또는 실행 안됨)")

        print(f"  - Ping 결과 요약: {final_state.get('ping_summary')}")
        
        print("\n--- 상세 원시 출력 (필요시 주석 해제) ---")
        if final_state.get('raw_ping_output'):
            print(final_state.get('raw_ping_output'))
        else:
            print("(원시 출력 없음)")

    except Exception as e:
        print(f"💥 그래프 실행 중 예외 발생: {e}")
        print("   (ping 유틸리티가 설치되어 있는지, ShellTool 실행 시 터미널 확인이 필요한지, OpenAI API 키 상태 등을 확인하세요.)")



📢 8.8.8.8에 대한 Ping 테스트를 시작합니다...
   확인 의도: 이 호스트가 살아있는지 확인하기 위해 6번만 ping을 보내줘.
---
--- NODE: Ping 명령어 생성 및 검증 중... ---
LLM에게 Ping 명령어 생성 요청 (IP: 8.8.8.8, 의도: 이 호스트가 살아있는지 확인하기 위해 6번만 ping을 보내줘.)
LLM 생성 원본 명령어: 'ping -c 6 8.8.8.8'
기본 검증 통과: 'ping -c 6 8.8.8.8'
--- 조건: Ping 명령어 생성/검증 성공, Ping 실행으로 ---
--- NODE: Ping 실행 중... ---
실행 명령어: ping -c 6 8.8.8.8
Executing command:
 ping -c 6 8.8.8.8
--- 조건: Ping 실행 완료 (성공/실패 무관), 결과 파싱으로 ---
--- NODE: Ping 결과 파싱 중... ---
파싱 결과: is_alive=True, summary='Host is alive (0% packet loss).'

---
📊 최종 Ping 테스트 결과:
  - 대상 IP: 8.8.8.8
  - 확인 의도: 이 호스트가 살아있는지 확인하기 위해 6번만 ping을 보내줘.
  - LLM 생성 원본 명령어: ping -c 6 8.8.8.8
  - 실행된 (검증된) 명령어: ping -c 6 8.8.8.8
  - 호스트 생존 여부: 🟢 살아있음 (응답)
  - Ping 결과 요약: Host is alive (0% packet loss).

--- 상세 원시 출력 (필요시 주석 해제) ---
PING 8.8.8.8 (8.8.8.8): 56 data bytes
64 bytes from 8.8.8.8: icmp_seq=0 ttl=112 time=43.737 ms
64 bytes from 8.8.8.8: icmp_seq=1 ttl=112 time=47.044 ms
64 bytes from 8.8.8.8: icmp_seq=2 ttl=112 time=47.7